In [ ]:
for Windows users:- https://chatgpt.com/canvas/shared/67f2ecf551d881918d2bb9093e65a4f9


You should see a message that says "Ollama is running."

If you don’t see that message, open a new Terminal (on Mac) or PowerShell (on Windows), and run the following command:

nginx
Copy
Edit
ollama serve
Then, in a separate Terminal or PowerShell window, run:

nginx
Copy
Edit
ollama pull llama3.2

Visit Ollama :- https://ollama.com/
Once everything is set up, the Ollama server should automatically start running on your machine.

To check, open your browser and go to:
http://localhost:11434/
On a PC, start a Command prompt / Powershell (Press Win + R, type `cmd`, and press Enter). 
On a Mac, start a Terminal (Applications > Utilities > Terminal).

In [18]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
print("hello")

hello


## Use this if you have issues 
from dotenv import load_dotenv
import os

# Load the environment variables and enable verbose output
load_dotenv(dotenv_path='C:\\Users\\rakad\\OneDrive\\Documents\\AgenticAI\\git\\rd_LLM_session1\\.env', verbose=True)

# Check if the API_KEY is loaded
api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    print(f"API_KEY is {api_key}")
else:
    print("API_KEY is not set.")
#print(api_key)




In [21]:
# Load environment variables in a file called .env
#

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key. Sometimes it shows error while copying it and hence

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")
   # print (api_key) 


API key found and looks good so far!


In [22]:
#Lets see whether we can do a quick call to a foundation Model 
openai = OpenAI()
# To give you a preview -- calling OpenAI with these messages is this easy. 

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)


Hello! It's great to hear from you! How can I assist you today?


In [23]:
#let's do a scraping on webpage
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [24]:
# Let's try one out. Change the website and add print statements to follow along.

cms = Website("https://www.cms.gov/medicare/quality/nursing-home-improvement/quality-measures")
print(cms.title)
#print(cms.text)

Quality Measures | CMS


In [26]:
# models 

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. I am running a few minutes late; my previous meeting is running over.Respond in markdown."

In [27]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [28]:
print(user_prompt_for(cms))

You are looking at a website titled Quality Measures | CMS
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main content
Centers for Medicare & Medicaid Services
CMS Newsroom
Header
About CMS
Newsroom
Data & Research
Search CMS.gov
Search
Search
Popular terms
Physician Fee Schedule
Local Coverage Determination
Medically Unlikely Edits
Telehealth
Covid-19
CMS.gov main menu
Medicare
Back to main menu
section title h2
Enrollment & renewal
Back to
menu
section title h3
Original Medicare (Part A and B) Eligibility and Enrollment
Annual Medicare Participation Announcement
Providers & suppliers
Medicare Managed Care Eligibility and Enrollment
Part D Eligibility and Enrollment
Health plans
Coverage
Back to
menu
section title h3
Coverage Determination Process
Medicare Coverage Database
Approved facilities, trials, & registries
Telehealth
Medicare Summary Notice
Prescri

In [29]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [30]:
messages_for(cms)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. I am running a few minutes late; my previous meeting is running over.Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Quality Measures | CMS\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to main content\nCenters for Medicare & Medicaid Services\nCMS Newsroom\nHeader\nAbout CMS\nNewsroom\nData & Research\nSearch CMS.gov\nSearch\nSearch\nPopular terms\nPhysician Fee Schedule\nLocal Coverage Determination\nMedically Unlikely Edits\nTelehealth\nCovid-19\nCMS.gov main menu\nMedicare\nBack to main menu\nsection title h2\nEnrollment & renewal\nBack to\nmenu\nsection title h3\nOriginal Medicare (Part A and B) Eligibility and Enrollment\nAnnual Medicare 

In [31]:

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [32]:
summarize("https://www.cms.gov/medicare/quality/nursing-home-improvement/quality-measures")

"# Summary of Quality Measures | CMS Website\n\nThe **Quality Measures | CMS** website serves as a comprehensive resource on quality measures related to healthcare services under the Centers for Medicare & Medicaid Services (CMS). The site outlines various quality improvement programs, including the Nursing Home Quality Initiative (NHQI), which focuses on enhancing care in nursing homes and ensuring resident safety.\n\n## Key Sections and Features:\n- **Quality Campaigns and Programs**: Information on various quality initiatives such as the Inpatient Rehabilitation Facility Quality Reporting Program, Home Health Quality Reporting Program, and the ESRD Quality Incentive Program.\n- **Quality Measures Resource**: Access to manuals and technical specifications like the Minimum Data Set (MDS) for nursing homes.\n- **Data Catalog**: A new Provider Data Catalog (PDC) has been introduced, facilitating easier access to public quality measure data.\n  \n## Recent Updates:\n- **December 23, 2024

In [35]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://www.cms.gov/medicare/quality/nursing-home-improvement/quality-measures")

# Quality Measures | CMS - Summary

The **Quality Measures** section of the **Centers for Medicare & Medicaid Services (CMS)** website provides comprehensive information about assessing and improving the quality of care in nursing homes and other healthcare facilities. It focuses on various quality initiatives, including:

- **Nursing Home Quality Improvement**: Programs aimed at enhancing care quality.
- **Quality Reporting Programs**: Detailed specifications for quality measures assessed using the Minimum Data Set (MDS).
- **Public Reporting Tools**: Resources for healthcare providers, including the Nursing Home Care Compare.

## Notable Updates
### Recent Announcements
1. **December 23, 2024**: Introduction of the MDS 3.0 Quality Measures User's Manual v17.0, providing major updates in quality measure specifications, particularly regarding MDS Section GG.
   
2. **July 1, 2024**: Availability of the MDS 3.0 Quality Measures User's Manual v16.0, updating risk adjustment details for certain Quality Measures effective October 1, 2023.

3. **September 21, 2023**: Announcement regarding the transition from MDS Section G to Section GG, impacting related quality measures. Several measures will be frozen for updates until January 2025 to align with this change.

### Current Quality Measures Highlighted
- **Short Stay Measures**: Include metrics like hospital readmissions and emergency department visits post-nursing home admission.
- **Long Stay Measures**: Monitor high-risk residents, medication usage, fall incidents, and overall functional status changes.

The website serves as a crucial resource for healthcare providers and stakeholders, ensuring transparency and accountability in nursing home care quality. Users can access detailed manuals, specifications, and important updates through the Downloads section.

Now the Home work part I guess :-D . By now you have downloaded the ollama part 
and so try any of the model part 
No Charges 